In [ ]:
# This snippet should take a dataframe in, and produce an xml compatible with upload into the KLASS-webgui
# It outputs to the jupyter-folder of this script.
# 
# See documentation here:
# - https://wiki.ssb.no/display/KP/Eksport+og+import+Excel
# Upload happens in
# - https://i.ssb.no/klass/admin/klassui
# Take a look at other examples by downloading the xmls from the KLASS UI
# 

In [ ]:
import pandas as pd

In [ ]:
# Example data into a pandas dataframe
colm = ['kode', 'navn_bokmål', 'navn_engelsk', 'kilde_kode', 'forelder']
rws = [['1', 'Ugift', 'Unmarried', 'a', None], ['2', 'Gift', 'Married', 'b', None]]

# Create dataframe
eks = pd.DataFrame(rws, columns = colm)
print(eks)

In [ ]:
#Function definition for writing content of xml

def to_klass(df: 'dataframe', filename = None, mode = "variant"):
    """
    Parameters:
    df (dataframe): Dataframe into first argument "to_klass"-function.
    filename (path): Path to write XML-file to. If not defined, no xml-file will be written.
    mode (str): What kind of "kodeverk" is being encoded. "variant", "version", "korrespondansetabell" etc.
    ### NOT ALL MODES HAVE BEEN CODED FOR YET ###
 
    Returns:
    str: output passed into xml-file
    writes XML file in root folder of this script

    """
    
    # There are most likely missing translations here for other things than "version"    
    ### UNFINISHED ###
    if (mode == 'version'):
        norsk_mode = 'versjon'
        start = "".join(['<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<ns1:', norsk_mode ,' xmlns:ns1="http://klass.ssb.no/', mode, '">\n'])
        element = "element"
        end = '\n</ns1:' + norsk_mode + '>'
    elif (mode == 'korrespondansetabell'):
        norsk_mode = 'Korrespondanse'
        start = '<Korrespondansetabell xmlns="http://klass.ssb.no/correspondenceTable" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" '
        start = start + 'xsi:schemaLocation="http://klass.ssb.no/correspondenceTable https://i.ssb.no/klass/admin/schemas/correspondenceTable.xsd">\n'
        element = 'Korrespondanse'
        end = '\n</ns1:Korrespondansetabell>'
    elif (mode == 'variant'):
        norsk_mode = 'variant'
        start = "".join(['<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<ns1:', norsk_mode ,' xmlns:ns1="http://klass.ssb.no/', mode, '">\n'])
        element = "element"
        end = '\n</ns1:' + norsk_mode + '>'
    else:
        norsk_mode = mode
        start = "".join(['<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<ns1:', norsk_mode ,' xmlns:ns1="http://klass.ssb.no/', mode, '">\n'])
        element = "element"
        end = '\n</ns1:' + norsk_mode + '>'
    
    ### The "ns1"-kode is added by Excel when doing the roundtrip, unsure if this is necessary.
    # Start building output
    
    def row_to_xml(row):
        """Returns xml-formatted row
        Parameters:
        row (dataframe row): Row of dataframe to operate on
        
        Returns:
        List of lines for XML, with content from the row in question        
        """
                
        xml = ['\t<ns1:' + element + '>']
        # Inserting content per row in dataframe
        for i, col_name in enumerate(row.index):
            cont = '\t\t'
            # Element is formatted differently when empty
            if row.iloc[i] is not None:
                cont = cont + '<ns1:{0}>{1}'.format(col_name, row.iloc[i])
                cont = cont + '</ns1:{0}>'.format(col_name, row.iloc[i])
            else: 
                cont = cont + '<ns1:{0}/>'.format(col_name, row.iloc[i])
            # insert content for cell in list
            xml.append(cont)
        # Finishing up the row
        xml.append('\t</ns1:' + element +'>')
        return '\n'.join(xml)

    # Apply the function to do per row on the whole dataframe
    res = '\n'.join(df.apply(row_to_xml, axis=1))
    # Combine all the pieces to complete output
    res = "".join([start, res, end])
    
    #If function is not passed filename, we want to return and exit    
    if filename is None:
        return res
    #If filename is passed, try writing a xml with content
    with open(filename, 'w') as f:
        f.write(res)
        return res

In [ ]:
#We assign the result of the function to an attribute of the dataframe
#The function runs at the same time, and will output an xml if filename is passed
#eks.to_klass = to_klass(eks, filename = './klass_eksempel.xml', mode="version")
eks.klass_xml = to_klass(eks, filename = './klass_eksempel.xml', mode="version")

#Returning values will not be shown, unless we print it
print(eks.klass_xml)